In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

import matplotlib.gridspec as gridspec
import re
from scipy.signal import argrelextrema as extrema
from matplotlib import animation, rc
from scipy.interpolate import griddata
from sys import exit
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon as pPolygon
import matplotlib as mpl
from shapely.geometry import MultiPoint, Point, Polygon
from scipy.spatial import Voronoi
from sys import exit
from scipy.ndimage.morphology import binary_erosion
from scipy.spatial import Voronoi
from skimage import draw
from sklearn.neighbors import KDTree
import pandas as pd

<ipython-input-2-7767fcb41197>:17: DeprecationWarning: Please use `binary_erosion` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  from scipy.ndimage.morphology import binary_erosion


In [3]:
def points_add():
    points = np.array([\
        [255.  , 255.  ], [297.5 , 255.  ], [276.25, 291.81], [233.75, 291.81], [212.5 , 255.  ], [233.75, 218.19], [276.25, 218.19], [340.  , 255.  ], [328.61, 297.5 ], [297.5 , 328.61], \
        [255.  , 340.  ], [212.5 , 328.61], [181.39, 297.5 ], [170.  , 255.  ], [181.39, 212.5 ], [328.61, 212.5 ], [382.5 , 255.  ], [374.81, 298.61], [352.67, 336.96], [318.75, 365.42], \
        [277.14, 380.56], [232.86, 380.56], [191.25, 365.42], [157.33, 336.96], [135.19, 298.61], [127.5 , 255.  ], [135.19, 211.39], [374.81, 211.39], [425.  , 255.  ], [419.21, 299.  ], \
        [402.22, 340.  ], [375.21, 375.21], [340.  , 402.22], [299.  , 419.21], [255.  , 425.  ], [211.  , 419.21], [170.  , 402.22], [134.79, 375.21], [107.78, 340.  ], [ 90.79, 299.  ], \
        [ 85.  , 255.  ], [ 90.79, 211.  ], [419.21, 211.  ], [467.5 , 255.  ], [462.86, 299.18], [449.13, 341.43], [426.92, 379.9 ], [397.19, 412.92], [361.25, 439.03], [320.67, 457.1 ], \
        [277.21, 466.34], [232.79, 466.34], [189.33, 457.1 ], [148.75, 439.03], [112.81, 412.92], [ 83.08, 379.9 ], [ 60.87, 341.43], [ 47.14, 299.18], [ 42.5 , 255.  ], [ 47.14, 210.82], \
        [462.86, 210.82]])

    ppoints = np.array([\
        [ 0.   ,  0.   ],\
        [ 2.   ,  0.   ], [ 2.   ,  1.047], [ 2.   ,  2.094], [ 2.   ,  3.142], [ 2.   ,  4.189], [ 2.   ,  5.236],\
        [ 4.   ,  0.   ], [ 4.   ,  0.524], [ 4.   ,  1.047], [ 4.   ,  1.571], [ 4.   ,  2.094], [ 4.   ,  2.618], [ 4.   ,  3.142], [ 4.   ,  3.668], [ 4.   ,  5.764],\
        [ 6.   ,  0.   ], [ 6.   ,  0.349], [ 6.   ,  0.698], [ 6.   ,  1.047], [ 6.   ,  1.396], [ 6.   ,  1.745], [ 6.   ,  2.094], [ 6.   ,  2.443], [ 6.   ,  2.793], [ 6.   ,  3.142], [ 6.   ,  3.491], [ 6.   ,  5.933],\
        [ 8.   ,  0.   ], [ 8.   ,  0.262], [ 8.   ,  0.524], [ 8.   ,  0.785], [ 8.   ,  1.047], [ 8.   ,  1.309], [ 8.   ,  1.571], [ 8.   ,  1.833], [ 8.   ,  2.094], [ 8.   ,  2.356], [ 8.   ,  2.618], [ 8.   ,  2.88 ], [ 8.   ,  3.142], [ 8.   ,  3.404], [ 8.   ,  6.022],\
        [10.   ,  0.   ], [10.   ,  0.209], [10.   ,  0.419], [10.   ,  0.628], [10.   ,  0.838], [10.   ,  1.047], [10.   ,  1.257], [10.   ,  1.466], [10.   ,  1.676], [10.   ,  1.885], [10.   ,  2.094], [10.   ,  2.304], [10.   ,  2.513], [10.   ,  2.723], [10.   ,  2.932], [10.   ,  3.142], [10.   ,  3.351], [10.   ,  6.075]])
    return points, ppoints


"""
plt.plot(points[:,0], points[:,1], 'ro', alpha = 0.0)
for i in range(points.shape[0]):
    plt.text(points[i,0], points[i,1], str(i))

plt.show()
"""

"\nplt.plot(points[:,0], points[:,1], 'ro', alpha = 0.0)\nfor i in range(points.shape[0]):\n    plt.text(points[i,0], points[i,1], str(i))\n\nplt.show()\n"

In [8]:
def polygonize_by_nearest_neighbor(pp):
    # start with first index
    pp_new = np.zeros_like(pp)
    pp_new[0] = pp[0]
    p_current_idx = 0

    tree = KDTree(pp)   #KDTree-

    for i in range(len(pp) - 1):

        nearest_dist, nearest_idx = tree.query([pp[p_current_idx]], k=4)  # k1 = identity
        nearest_idx = nearest_idx[0]

        for min_idx in nearest_idx[1:]:  # skip the first point (will be zero for same pixel)
            if not pp[min_idx].tolist() in pp_new.tolist():  # make sure it's not already in the list
                pp_new[i + 1] = pp[min_idx]
                p_current_idx = min_idx
                break

    pp_new[-1] = pp[0]
    return pp_new

In [7]:
def voronoi_finite_polygons_2d(vor, radius=None):
    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")

    new_regions = []
    new_vertices = vor.vertices.tolist()
    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max()

    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]

        if all(v >= 0 for v in vertices):
            new_regions.append(vertices)
            continue

        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]
        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                continue

            t = vor.points[p2] - vor.points[p1] # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal

            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius

            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())

        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:,1] - c[1], vs[:,0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]
        new_regions.append(new_region.tolist())

    return new_regions, np.asarray(new_vertices)

In [6]:
def gen_polys(points, ppoints):
    #generates a circular mask
    side_len = 500
    rad = 235
    mask = np.zeros(shape=(side_len, side_len))
    rr, cc = draw.ellipse(side_len/2, side_len/2+5, r_radius=rad, c_radius=rad, shape=mask.shape)
    mask[rr, cc] = 1
    mask[:int(side_len/2)-1,:] = 0

    #makes a polygon from the mask perimeter
    #se = get_circular_se(radius=1)
    #contour = mask - binary_erosion(mask, structure=se)
    contour = mask - binary_erosion(mask)
    pixels_mask = np.array(np.where(contour==1)[::-1]).T
    polygon = polygonize_by_nearest_neighbor(pixels_mask)
    polygon = Polygon(polygon)
    new_points, new_ppoints = [], []
    for i, point in enumerate(points):
        if polygon.contains(Point(point)):
            new_points.append([p*(1+np.random.uniform(-0.00,0.00)) for p in point])
            new_ppoints.append(ppoints[i])
    new_points = np.array(new_points)
    new_ppoints = np.array(new_ppoints)
    #performs voronoi tesselation
    #if len(points) > 3: #otherwise the tesselation won't work

    vor = Voronoi(new_points)
    regions, vertices = voronoi_finite_polygons_2d(vor)

    #clips tesselation to the mask
    new_vertices = []
    for region in regions:
        poly_reg = vertices[region]
        shape = list(poly_reg.shape)
        shape[0] += 1
        p = Polygon(np.append(poly_reg, poly_reg[0]).reshape(*shape)).intersection(polygon)
        poly = (np.array(p.exterior.coords)).tolist()
        new_vertices.append(poly)

    return new_vertices

In [1]:
class Model2d:
    def __init__(self):
        self.points = None
        self.polys = None
        self.neib_mx = None
        self.sp_pts = {}    # Dict of special points
        return None

    def init_2d_dome_model(self, neib_cut=55):
        self.points, ppoints = points_add()
        self.neib_mx = get_neib_mx(self.points, cutoff=neib_cut)
        self.polys = gen_polys(self.points, ppoints)

    def add_sp_pts(self, ids=[], name=None):
        self.sp_pts[name] = ids
        return 0

